In [ ]:
import numpy as np
import os
import math
import pandas as pd
import scipy.stats as stat
from scipy.stats import qmc
from scipy.stats import norm
import scipy.interpolate
import statistics
from statistics import NormalDist

import warnings
warnings.filterwarnings("ignore")

# 👉 <a id = "top">Table of Contents</a> 👈 

# [1. Download Data](#p1)

# [2. Preprocess Swap Data](#p2)

# [3. Preprocess Stock Data](#p3)

# [4. Parametric VaR Model](#p4)

### [4a. Parametric VaR for stocks](#p4a)

### [4b. Parametric VaR for swap](#p4b)

### [4c. Parametric VaR for portfolio of stocks and swap](#p4c)

# [5. Monte Carlo VaR Model](#p5)

### [5a. Monte Carlo Full Revaluation](#p5a)

### [5b. Monte Carlo Sensitivity Based Revaluation](#p5b)

# [6. Historical VaR Model](#p6)

### [6a. Historical Full Revaluation](#p6a)

### [6b. Historical Sensitivity Based Revaluation](#p6b)

# <a id = "p1">1.</a>  <font color = "green"> Download Data </font>  [back to table of contents](#top)

In [2]:
SOFR = pd.read_excel("hist_data.xlsm",
                     sheet_name="SofrCurve")

SOFR 

,Tenor,T,2022-10-31 00:00:00,2022-11-01 00:00:00,2022-11-02 00:00:00,2022-11-03 00:00:00,2022-11-04 00:00:00,2022-11-07 00:00:00,2022-11-08 00:00:00,2022-11-09 00:00:00,...,2023-10-17 00:00:00,2023-10-18 00:00:00,2023-10-19 00:00:00,2023-10-20 00:00:00,2023-10-23 00:00:00,2023-10-24 00:00:00,2023-10-25 00:00:00,2023-10-26 00:00:00,2023-10-27 00:00:00,2023-10-30 00:00:00
0,1D,0.002778,0.039191,0.039604,0.039948,0.040389,0.045965,0.040224,0.040339,0.040373,...,0.052964,0.053038,0.053050,0.053072,0.053090,0.053105,0.052981,0.053047,0.052989,0.052967
1,1M,0.083333,0.038721,0.039023,0.039286,0.039585,0.042343,0.039258,0.039211,0.039193,...,0.053114,0.053102,0.053086,0.053046,0.053077,0.053084,0.053057,0.053089,0.053040,0.053053
2,2M,0.166667,0.038670,0.038886,0.039100,0.039350,0.038795,0.039460,0.039420,0.039483,...,0.053374,0.053284,0.053226,0.053128,0.053191,0.053202,0.053259,0.053218,0.053185,0.053265
3,3M,0.250000,0.040536,0.040725,0.040852,0.041154,0.040611,0.041424,0.041406,0.041445,...,0.053666,0.053561,0.053454,0.053323,0.053404,0.053424,0.053490,0.053414,0.053368,0.053476
4,6M,0.500000,0.044577,0.044849,0.044884,0.045281,0.045212,0.045455,0.045334,0.045299,...,0.054078,0.053989,0.053758,0.053546,0.053638,0.053690,0.053749,0.053567,0.053486,0.053612
5,9M,0.750000,0.046004,0.046448,0.046580,0.047107,0.046752,0.047189,0.046941,0.046883,...,0.053831,0.053753,0.053427,0.053126,0.053211,0.053293,0.053393,0.053089,0.052991,0.053120
6,1Y,1.000000,0.046449,0.046970,0.047203,0.047894,0.047500,0.047929,0.047589,0.047479,...,0.053225,0.053163,0.052750,0.052348,0.052389,0.052503,0.052653,0.052243,0.052115,0.052245
7,2Y,2.000000,0.044583,0.045022,0.045496,0.046594,0.046097,0.046633,0.046046,0.045585,...,0.049497,0.049611,0.049204,0.048366,0.048027,0.048399,0.048791,0.048044,0.047758,0.047904
8,3Y,3.000000,0.042002,0.042344,0.042749,0.043833,0.043385,0.043884,0.043209,0.042645,...,0.046978,0.047304,0.047169,0.046151,0.045582,0.045999,0.046595,0.045645,0.045284,0.045429
9,4Y,4.000000,0.040318,0.040614,0.040868,0.041825,0.041503,0.041916,0.041182,0.040704,...,0.045605,0.046034,0.046195,0.045156,0.044508,0.044850,0.045594,0.044538,0.044200,0.044345


In [3]:
AAPL = pd.read_excel("hist_data.xlsm",
                     sheet_name="AAPL",
                     index_col=0)

AAPL.rename(columns = {"Adj Close":"AAPL"},
            inplace=True)

AAPL

,AAPL
Date,
2022-10-31,152.041122
2022-11-01,149.373917
2022-11-02,143.801514
2022-11-03,137.703613
2022-11-04,137.435455
...,...
2023-10-24,172.991058
2023-10-25,170.657135
2023-10-26,166.458023


In [4]:
MSFT = pd.read_excel("hist_data.xlsm",
                     sheet_name="MSFT",
                     index_col=0)

MSFT.rename(columns = {"Adj Close":"MSFT"},
            inplace=True)

MSFT

,MSFT
Date,
2022-10-31,229.443207
2022-11-01,225.529037
2022-11-02,217.552444
2022-11-03,211.770157
2022-11-04,218.827515
...,...
2023-10-24,329.860504
2023-10-25,339.979980
2023-10-26,327.225861


In [5]:
FORD = pd.read_excel("hist_data.xlsm",
                     sheet_name="F",
                     index_col=0)

FORD.rename(columns = {"Adj Close":"FORD"},
            inplace=True)

FORD

,FORD
Date,
2022-10-31,11.974380
2022-11-01,12.001248
2022-11-02,11.696740
2022-11-03,11.875863
2022-11-04,12.099767
...,...
2023-10-24,11.215128
2023-10-25,11.362824
2023-10-26,11.175742


In [6]:
BAC = pd.read_excel("hist_data.xlsm",
                     sheet_name="BAC",
                     index_col=0)

BAC.rename(columns = {"Adj Close":"BAC"},
            inplace=True)

BAC

,BAC
Date,
2022-10-31,34.748581
2022-11-01,34.902847
2022-11-02,34.796783
2022-11-03,34.603954
2022-11-04,35.471703
...,...
2023-10-24,25.268324
2023-10-25,25.347691
2023-10-26,25.913177


# <a id = "p2">2.</a>  <font color = "green"> Preprocess Swap Data </font>  [back to table of contents](#top)

In [7]:
swap_notional = 100000000
fixed_leg_interest = 0.042

In [8]:
SOFR = SOFR.T.iloc[1:]

SOFR = SOFR.rename(columns=SOFR.iloc[0]).iloc[1:]

SOFR

,0.002778,0.083333,0.166667,0.250000,0.500000,0.750000,1.000000,2.000000,3.000000,4.000000,...,15.000000,16.000000,17.000000,18.000000,19.000000,20.000000,25.000000,30.000000,35.000000,40.000000
2022-10-31 00:00:00,0.039191,0.038721,0.03867,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.03235,0.030552,0.028708
2022-11-01 00:00:00,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.04697,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.03601,0.035678,0.033645,0.031979,0.030238,0.028478
2022-11-02 00:00:00,0.039948,0.039286,0.0391,0.040852,0.044884,0.04658,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
2022-11-03 00:00:00,0.040389,0.039585,0.03935,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
2022-11-04 00:00:00,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.0475,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.03738,0.037152,0.03687,0.036534,0.034424,0.032558,0.030723,0.028933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24 00:00:00,0.053105,0.053084,0.053202,0.053424,0.05369,0.053293,0.052503,0.048399,0.045999,0.04485,...,0.044093,0.044056,0.043972,0.043841,0.043663,0.043437,0.041874,0.040238,0.038438,0.036493
2023-10-25 00:00:00,0.052981,0.053057,0.053259,0.05349,0.053749,0.053393,0.052653,0.048791,0.046595,0.045594,...,0.045289,0.045268,0.045198,0.045078,0.044906,0.044683,0.043076,0.041358,0.039564,0.037641
2023-10-26 00:00:00,0.053047,0.053089,0.053218,0.053414,0.053567,0.053089,0.052243,0.048044,0.045645,0.044538,...,0.044269,0.044244,0.044171,0.044049,0.043882,0.043668,0.042182,0.040589,0.03876,0.036764
2023-10-27 00:00:00,0.052989,0.05304,0.053185,0.053368,0.053486,0.052991,0.052115,0.047758,0.045284,0.0442,...,0.04446,0.044461,0.044411,0.044309,0.044157,0.043955,0.042508,0.040985,0.039167,0.037148


In [9]:
# Filter for 1-10Y rates
SOFR  = SOFR [[i for i in range(1, 11)]]

SOFR

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
2022-10-31 00:00:00,0.046449,0.044583,0.042002,0.040318,0.039297,0.038534,0.037979,0.037648,0.037465,0.037353
2022-11-01 00:00:00,0.04697,0.045022,0.042344,0.040614,0.039522,0.038658,0.038007,0.037605,0.037368,0.037209
2022-11-02 00:00:00,0.047203,0.045496,0.042749,0.040868,0.039675,0.038776,0.038117,0.037704,0.037458,0.0373
2022-11-03 00:00:00,0.047894,0.046594,0.043833,0.041825,0.040507,0.039537,0.03882,0.038328,0.038002,0.037796
2022-11-04 00:00:00,0.0475,0.046097,0.043385,0.041503,0.040303,0.039421,0.038782,0.038373,0.038128,0.037988
...,...,...,...,...,...,...,...,...,...,...
2023-10-24 00:00:00,0.052503,0.048399,0.045999,0.04485,0.044305,0.044036,0.043889,0.043816,0.043798,0.043826
2023-10-25 00:00:00,0.052653,0.048791,0.046595,0.045594,0.0452,0.045017,0.044911,0.044867,0.044876,0.044926
2023-10-26 00:00:00,0.052243,0.048044,0.045645,0.044538,0.044086,0.043893,0.043808,0.043795,0.043829,0.043898
2023-10-27 00:00:00,0.052115,0.047758,0.045284,0.0442,0.043762,0.043636,0.043643,0.043702,0.043793,0.043908


In [10]:
DF = SOFR.copy()

# compute disocunt factors
for i in range(len(DF.columns)):
    DF[DF.columns[i]] = np.exp(- DF[DF.columns[i]].astype(float) * (i + 1))

In [11]:
DF

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
2022-10-31 00:00:00,0.954613,0.914694,0.881609,0.851061,0.821614,0.793577,0.766553,0.739942,0.713778,0.688304
2022-11-01 00:00:00,0.954116,0.913891,0.880705,0.850053,0.820689,0.792987,0.766399,0.740196,0.714403,0.689294
2022-11-02 00:00:00,0.953893,0.913025,0.879637,0.849189,0.820063,0.792425,0.765814,0.739612,0.713825,0.688664
2022-11-03 00:00:00,0.953235,0.911023,0.876780,0.845945,0.816658,0.788816,0.762050,0.735930,0.710334,0.685261
2022-11-04 00:00:00,0.953611,0.911928,0.877959,0.847037,0.817491,0.789363,0.762252,0.735662,0.709530,0.683946
...,...,...,...,...,...,...,...,...,...,...
2023-10-24 00:00:00,0.948851,0.907740,0.871102,0.835770,0.801294,0.767810,0.735488,0.704315,0.674231,0.645160
2023-10-25 00:00:00,0.948709,0.907028,0.869546,0.833289,0.797719,0.763303,0.730246,0.698417,0.667725,0.638101
2023-10-26 00:00:00,0.949098,0.908384,0.872027,0.836814,0.802175,0.768469,0.735902,0.704436,0.674042,0.644694
2023-10-27 00:00:00,0.949220,0.908905,0.872971,0.837948,0.803475,0.769654,0.736756,0.704958,0.674264,0.644626


In [12]:
def PV_payer_swap(discount_factors):
    # payer swap PV = notional + (- notional * tenor DF - discounted CF of fixed leg)
    float_leg =  (1 - discount_factors[9])
    fix_leg = fixed_leg_interest * discount_factors.sum()
    payer_swap_PV  = swap_notional * (float_leg - fix_leg)

    return payer_swap_PV

In [13]:
DF.iloc[-1, ]

1.0     0.949097
2.0     0.908638
3.0     0.872591
4.0     0.837462
5.0     0.802808
6.0     0.768923
7.0     0.736051
8.0     0.704247
9.0     0.673524
10.0    0.643887
Name: 2023-10-30 00:00:00, dtype: float64

In [14]:
# Get the discount factors for today
today_DF = DF.iloc[-1, ].to_numpy()
today_DF

array([0.94909668, 0.90863809, 0.87259128, 0.83746233, 0.80280786,
       0.76892333, 0.7360513 , 0.70424673, 0.6735244 , 0.64388735])

In [15]:
# Compute the current value of the swap

payer_swap_today = PV_payer_swap(today_DF)
payer_swap_today

2442901.9998463253

# <a id = "p3">3.</a>  <font color = "green"> Preprocess Stock Data </font>  [back to table of contents](#top)

In [16]:
stocks_notional = 1000000

In [17]:
Stocks = pd.concat([AAPL,MSFT,FORD,BAC],axis = 1)

Stocks

,AAPL,MSFT,FORD,BAC
Date,,,,
2022-10-31,152.041122,229.443207,11.974380,34.748581
2022-11-01,149.373917,225.529037,12.001248,34.902847
2022-11-02,143.801514,217.552444,11.696740,34.796783
2022-11-03,137.703613,211.770157,11.875863,34.603954
2022-11-04,137.435455,218.827515,12.099767,35.471703
...,...,...,...,...
2023-10-24,172.991058,329.860504,11.215128,25.268324
2023-10-25,170.657135,339.979980,11.362824,25.347691
2023-10-26,166.458023,327.225861,11.175742,25.913177


In [18]:
curr_stocks_price = Stocks.iloc[-1].to_numpy()
curr_stocks_price

array([169.849197, 336.62677 ,   9.62    ,  25.486582])

In [19]:
stocks_total_value = stocks_notional * Stocks.shape[1]
stocks_total_value

4000000

In [20]:
# initial value of holdings
stocks_holdings_value = np.array([stocks_notional] * Stocks.shape[1])
stocks_holdings_value

array([1000000, 1000000, 1000000, 1000000])

# <a id = "p4">4.</a>  <font color = "green"> Parametric VaR Model </font>  [back to table of contents](#top)

In [21]:
VaR_percentile = 5

In [22]:
# z value
z = norm.ppf(VaR_percentile / 100)
z

-1.6448536269514729

### <a id = "p4a">4a.</a>  <font color = "green"> Parametric VaR for stocks </font>  [back to table of contents](#top)

In [23]:
# Compute stock returns
Stock_returns = Stocks.pct_change()
Stock_returns.dropna(inplace=True)
Stock_returns

,AAPL,MSFT,FORD,BAC
Date,,,,
2022-11-01,-0.017543,-0.017059,0.002244,0.004439
2022-11-02,-0.037305,-0.035368,-0.025373,-0.003039
2022-11-03,-0.042405,-0.026579,0.015314,-0.005542
2022-11-04,-0.001947,0.033326,0.018854,0.025077
2022-11-07,0.003902,0.029270,0.014064,0.005980
...,...,...,...,...
2023-10-24,0.002543,0.003674,-0.007840,-0.003911
2023-10-25,-0.013492,0.030678,0.013169,0.003141
2023-10-26,-0.024606,-0.037514,-0.016464,0.022309


In [24]:
# Stock returns statistics
Stocks_mean = Stock_returns.mean(axis = 0).to_numpy()
Stocks_std = Stock_returns.std(axis = 0).to_numpy()
Stocks_cov = Stock_returns[[col for col in Stock_returns.columns]].iloc[1:, ].cov().to_numpy()

In [25]:
Stocks_mean

array([ 0.000568  ,  0.0016922 , -0.00061889, -0.0011051 ])

In [26]:
Stocks_std

array([0.01587415, 0.01788419, 0.02253815, 0.01637649])

In [27]:
Stocks_cov

array([[2.51676920e-04, 1.82797488e-04, 1.35557917e-04, 7.60232547e-05],
       [1.82797488e-04, 3.19710313e-04, 1.34746553e-04, 7.05779219e-05],
       [1.35557917e-04, 1.34746553e-04, 5.09983110e-04, 1.69028950e-04],
       [7.60232547e-05, 7.05779219e-05, 1.69028950e-04, 2.69146481e-04]])

In [28]:
# parametric stock statistics
Para_stock_mean = (Stocks_mean * stocks_holdings_value).sum()
Para_stock_std = np.sqrt((stocks_holdings_value @ Stocks_cov) @ stocks_holdings_value.T)

In [29]:
# Parametric VaR for individual assets
Para_stock_VaR = abs(Para_stock_mean + z * Para_stock_std)
Para_stock_VaR

87858.12289544765

### <a id = "p4b">4b.</a>  <font color = "green"> Parametric VaR for swap </font>  [back to table of contents](#top)

In [30]:
bp_change = 0.0001

In [31]:
n_tenors = len(SOFR.columns)
n_tenors

10

In [32]:
payer_change = np.zeros(n_tenors)
payer_change

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [33]:
sofr_delta = SOFR.diff().dropna()
sofr_delta

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
2022-11-01 00:00:00,0.000521,0.000439,0.000342,0.000296,0.000225,0.000124,0.000029,-0.000043,-0.000097,-0.000144
2022-11-02 00:00:00,0.000234,0.000474,0.000404,0.000254,0.000153,0.000118,0.000109,0.000099,0.00009,0.000091
2022-11-03 00:00:00,0.00069,0.001098,0.001085,0.000957,0.000832,0.000761,0.000704,0.000624,0.000545,0.000495
2022-11-04 00:00:00,-0.000394,-0.000497,-0.000448,-0.000322,-0.000204,-0.000116,-0.000038,0.000045,0.000126,0.000192
2022-11-07 00:00:00,0.000429,0.000536,0.000499,0.000413,0.000332,0.000313,0.000328,0.000342,0.000345,0.000339
...,...,...,...,...,...,...,...,...,...,...
2023-10-24 00:00:00,0.000114,0.000371,0.000417,0.000343,0.000257,0.000165,0.000083,0.000032,0.0,-0.000027
2023-10-25 00:00:00,0.00015,0.000393,0.000596,0.000743,0.000894,0.000981,0.001022,0.001051,0.001077,0.0011
2023-10-26 00:00:00,-0.00041,-0.000747,-0.00095,-0.001055,-0.001114,-0.001124,-0.001102,-0.001073,-0.001046,-0.001028
2023-10-27 00:00:00,-0.000129,-0.000286,-0.000361,-0.000339,-0.000324,-0.000257,-0.000166,-0.000093,-0.000037,0.000011


In [34]:
# filter for rates today
sofr_today = SOFR.iloc[-1, ].to_numpy()
sofr_today

array([0.0522446062245015, 0.0479042005661318, 0.0454293376025913,
       0.0443447502563426, 0.0439279746825301, 0.043794001970382,
       0.0437793521534213, 0.0438283144768502, 0.0439145618623502,
       0.044023149444895], dtype=object)

In [35]:
for tenor in range(n_tenors):

    # add 1bp to the rate for that tenor
    pv01_sofr = np.zeros(len(sofr_today))

    for j in range(n_tenors):
        if j == tenor:
            pv01_sofr[j] = sofr_today[j] + bp_change
        else:
            pv01_sofr[j] = sofr_today[j]

    new_DF = np.zeros(len(pv01_sofr))

    # compute new discount factors
    for i in range(len(pv01_sofr)):
        new_DF[i] = np.exp(- pv01_sofr[i] * (i + 1))

    # compute PV01 change in swap value
    payer_change[tenor] = PV_payer_swap(new_DF) - payer_swap_today

In [36]:
payer_change

array([  398.6006767 ,   763.17967796,  1099.30010806,  1406.65535666,
        1685.47509868,  1937.10560866,  2163.23359438,  2365.32275674,
        2544.77690647, 67059.52625624])

In [37]:
# convert bp to rate %
pv01 = payer_change / bp_change
pv01

array([3.98600677e+06, 7.63179678e+06, 1.09930011e+07, 1.40665536e+07,
       1.68547510e+07, 1.93710561e+07, 2.16323359e+07, 2.36532276e+07,
       2.54477691e+07, 6.70595263e+08])

In [38]:
# mean change in rates
sofr_mean = sofr_delta.mean(axis = 0).to_numpy()

In [39]:
# std change in rates 
sofr_std = sofr_delta.std(axis = 0).to_numpy()

In [40]:
# covariance matrixs of rates
sofr_cov = (sofr_delta).cov(numeric_only = False).to_numpy()

In [41]:
# Parametric swap statistics
Para_swap_mean = (pv01 * sofr_mean).sum()
Para_swap_std = np.sqrt((pv01 @ sofr_cov) @ pv01.T)

In [42]:
Para_swap_mean

20927.25140634372

In [43]:
Para_swap_std

579882.9007355204

In [44]:
Para_swap_VaR = abs(Para_swap_mean + z * Para_swap_std)
Para_swap_VaR 

932895.2410756178

### <a id = "p4c">4c.</a>  <font color = "green"> Parametric VaR for portfolio of stocks and swap </font>  [back to table of contents](#top)

In [45]:
portfolio_value = np.append(pv01, stocks_holdings_value)
portfolio_value

array([3.98600677e+06, 7.63179678e+06, 1.09930011e+07, 1.40665536e+07,
       1.68547510e+07, 1.93710561e+07, 2.16323359e+07, 2.36532276e+07,
       2.54477691e+07, 6.70595263e+08, 1.00000000e+06, 1.00000000e+06,
       1.00000000e+06, 1.00000000e+06])

In [46]:
portfolio_risk_changes = pd.concat([sofr_delta, Stock_returns], axis = 1)
portfolio_risk_changes.dropna(inplace=True)
portfolio_risk_changes

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,AAPL,MSFT,FORD,BAC
2022-11-01 00:00:00,0.000521,0.000439,0.000342,0.000296,0.000225,0.000124,0.000029,-0.000043,-0.000097,-0.000144,-0.017543,-0.017059,0.002244,0.004439
2022-11-02 00:00:00,0.000234,0.000474,0.000404,0.000254,0.000153,0.000118,0.000109,0.000099,0.00009,0.000091,-0.037305,-0.035368,-0.025373,-0.003039
2022-11-03 00:00:00,0.00069,0.001098,0.001085,0.000957,0.000832,0.000761,0.000704,0.000624,0.000545,0.000495,-0.042405,-0.026579,0.015314,-0.005542
2022-11-04 00:00:00,-0.000394,-0.000497,-0.000448,-0.000322,-0.000204,-0.000116,-0.000038,0.000045,0.000126,0.000192,-0.001947,0.033326,0.018854,0.025077
2022-11-07 00:00:00,0.000429,0.000536,0.000499,0.000413,0.000332,0.000313,0.000328,0.000342,0.000345,0.000339,0.003902,0.029270,0.014064,0.005980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24 00:00:00,0.000114,0.000371,0.000417,0.000343,0.000257,0.000165,0.000083,0.000032,0.0,-0.000027,0.002543,0.003674,-0.007840,-0.003911
2023-10-25 00:00:00,0.00015,0.000393,0.000596,0.000743,0.000894,0.000981,0.001022,0.001051,0.001077,0.0011,-0.013492,0.030678,0.013169,0.003141
2023-10-26 00:00:00,-0.00041,-0.000747,-0.00095,-0.001055,-0.001114,-0.001124,-0.001102,-0.001073,-0.001046,-0.001028,-0.024606,-0.037514,-0.016464,0.022309
2023-10-27 00:00:00,-0.000129,-0.000286,-0.000361,-0.000339,-0.000324,-0.000257,-0.000166,-0.000093,-0.000037,0.000011,0.007969,0.005856,-0.122467,-0.036371


In [47]:
portfolio_risk_changes_cov = portfolio_risk_changes.cov(numeric_only = False)
portfolio_risk_changes_cov

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,AAPL,MSFT,FORD,BAC
1.0,6.447837e-07,6.911858e-07,6.110915e-07,5.551807e-07,5.020748e-07,4.582472e-07,4.257226e-07,4.011945e-07,3.807200e-07,3.611303e-07,-2.529316e-07,-0.000001,1.805932e-06,0.000005
2.0,6.911858e-07,8.359422e-07,7.888955e-07,7.365705e-07,6.805591e-07,6.314281e-07,5.922902e-07,5.606505e-07,5.332731e-07,5.076879e-07,-8.159149e-07,-0.000001,1.437525e-06,0.000005
3.0,6.110915e-07,7.888955e-07,7.766477e-07,7.408717e-07,6.965094e-07,6.552299e-07,6.203618e-07,5.907018e-07,5.642669e-07,5.396070e-07,-1.082691e-06,-0.000001,8.838852e-07,0.000004
4.0,5.551807e-07,7.365705e-07,7.408717e-07,7.191139e-07,6.844643e-07,6.502705e-07,6.205148e-07,5.942522e-07,5.702199e-07,5.475904e-07,-1.185816e-06,-0.000001,3.563960e-07,0.000003
5.0,5.020748e-07,6.805591e-07,6.965094e-07,6.844643e-07,6.588816e-07,6.316259e-07,6.066606e-07,5.838869e-07,5.626071e-07,5.423710e-07,-1.226989e-06,-0.000001,2.080452e-08,0.000003
6.0,4.582472e-07,6.314281e-07,6.552299e-07,6.502705e-07,6.316259e-07,6.108047e-07,5.908791e-07,5.715969e-07,5.528554e-07,5.347978e-07,-1.233524e-06,-0.000001,-2.249157e-07,0.000002
7.0,4.257226e-07,5.922902e-07,6.203618e-07,6.205148e-07,6.066606e-07,5.908791e-07,5.754697e-07,5.595023e-07,5.432066e-07,5.271667e-07,-1.249853e-06,-0.000001,-4.354106e-07,0.000002
8.0,4.011945e-07,5.606505e-07,5.907018e-07,5.942522e-07,5.838869e-07,5.715969e-07,5.595023e-07,5.464938e-07,5.327131e-07,5.186805e-07,-1.305490e-06,-0.000001,-6.332534e-07,0.000002
9.0,3.807200e-07,5.332731e-07,5.642669e-07,5.702199e-07,5.626071e-07,5.528554e-07,5.432066e-07,5.327131e-07,5.212784e-07,5.091422e-07,-1.371200e-06,-0.000001,-8.140452e-07,0.000002
10.0,3.611303e-07,5.076879e-07,5.396070e-07,5.475904e-07,5.423710e-07,5.347978e-07,5.271667e-07,5.186805e-07,5.091422e-07,4.986508e-07,-1.408930e-06,-0.000001,-9.599543e-07,0.000002


In [48]:
portfolio_risk_changes_mean = portfolio_risk_changes.mean().to_numpy()
portfolio_risk_changes_mean

array([1.834251998399881e-05, 6.5764790088367006e-06,
       6.707498381593196e-06, 1.188098061665042e-05,
       1.4895963619527458e-05, 1.734712467534517e-05,
       1.9652927819089913e-05, 2.134761056332942e-05,
       2.2617222309097604e-05, 2.3682717698658473e-05,
       0.00046080236288395884, 0.0016057696059283524,
       -0.0007383992089266441, -0.0011807395333843654], dtype=object)

In [49]:
portfolio_risk_changes_std = portfolio_risk_changes.std().to_numpy()
portfolio_risk_changes_std

array([0.0008029842446578996, 0.000914298760045892, 0.0008812761766376332,
       0.0008480058429180959, 0.0008117152398273903,
       0.0007815399602530434, 0.0007585971844732306,
       0.0007392521560341573, 0.0007219961450248529,
       0.0007061521131133847, 0.015885531523082908, 0.01792556766841346,
       0.02257701938687355, 0.016420595410625023], dtype=object)

In [50]:
Para_port_mean = portfolio_value @ portfolio_risk_changes_mean
Para_port_mean

18485.85162743529

In [51]:
Para_port_std = np.sqrt(portfolio_value @ portfolio_risk_changes_cov @ portfolio_value.T)
Para_port_std

580826.8238683383

In [52]:
Para_port_VaR = abs(Para_port_mean + z * Para_port_std)
Para_port_VaR 

936889.2562431052

In [53]:
print(f'Parametric Portfolio 95% VaR: ${round(Para_port_VaR, 2):,}')

Parametric Portfolio 95% VaR: $936,889.26


# <a id = "p5">5.</a>  <font color = "green"> Monte Carlo VaR Model </font>  [back to table of contents](#top)

In [54]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.Sobol.html

def simulate(dimension, power):
    simulation = qmc.Sobol(d = dimension, scramble = False)
    simulation = simulation.random_base2(m = power)
    simulation = np.delete(simulation, 0, axis = 0)
    print(f'{dimension} X {2 ** power:,} simulations generated')

    return simulation

In [55]:
# Simulate risk factor changes
portfolio_risk_changes_simulation = simulate(len(stocks_holdings_value) + len(SOFR.columns), 21)
portfolio_risk_changes_simulation

14 X 2,097,152 simulations generated


array([[5.00000000e-01, 5.00000000e-01, 5.00000000e-01, ...,
        5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [7.50000000e-01, 2.50000000e-01, 2.50000000e-01, ...,
        7.50000000e-01, 2.50000000e-01, 2.50000000e-01],
       [2.50000000e-01, 7.50000000e-01, 7.50000000e-01, ...,
        2.50000000e-01, 7.50000000e-01, 7.50000000e-01],
       ...,
       [7.50000477e-01, 7.81258106e-01, 1.57850742e-01, ...,
        2.19658375e-01, 8.03422928e-03, 3.02476406e-01],
       [5.00000477e-01, 3.12581062e-02, 9.07850742e-01, ...,
        4.69658375e-01, 7.58034229e-01, 5.52476406e-01],
       [4.76837158e-07, 5.31258106e-01, 4.07850742e-01, ...,
        9.69658375e-01, 2.58034229e-01, 5.24764061e-02]])

In [56]:
# compute correlation matrix and cholesky decomposition
portfolio_corr_matrix = portfolio_risk_changes.corr(numeric_only = False).to_numpy()
portfolio_corr_matrix

array([[ 1.        ,  0.9414552 ,  0.86354947,  0.81532068,  0.77029603,
         0.73019967,  0.69888939,  0.6758579 ,  0.65669504,  0.63688146,
        -0.0198287 , -0.06984395,  0.09961571,  0.34438553],
       [ 0.9414552 ,  1.        ,  0.97908256,  0.95000824,  0.91700994,
         0.88365869,  0.85395536,  0.82949064,  0.80784243,  0.78634004,
        -0.05617654, -0.08796492,  0.0696403 ,  0.30428368],
       [ 0.86354947,  0.97908256,  1.        ,  0.99136163,  0.97366879,
         0.9513284 ,  0.92794411,  0.90670006,  0.88682451,  0.86709616,
        -0.07733761, -0.09297457,  0.04442395,  0.25930777],
       [ 0.81532068,  0.95000824,  0.99136163,  1.        ,  0.99437049,
         0.98116943,  0.96458848,  0.94793681,  0.93134079,  0.91444745,
        -0.08802719, -0.09504394,  0.01861519,  0.22597258],
       [ 0.77029603,  0.91700994,  0.97366879,  0.99437049,  1.        ,
         0.99564623,  0.98521459,  0.97304384,  0.95998975,  0.94622517,
        -0.09515576, -0.09

In [57]:
portfolio_cholesky = np.linalg.cholesky(portfolio_corr_matrix)
portfolio_cholesky

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.9414552 ,  0.33713812,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.86354947,  0.49264506,  0.10762509,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.81532068,  0.54108492,  0.19260448,  0.07336767,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.77029603,  0.56893815,  0.26197879,  0.10944963,  0.04838696,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.  

In [58]:
# norm inverse to get normal distribution
portfolio_simulated_risk_changes = norm.ppf(portfolio_risk_changes_simulation, loc = 0, scale = 1)
portfolio_simulated_risk_changes

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.74489750e-01, -6.74489750e-01, -6.74489750e-01, ...,
         6.74489750e-01, -6.74489750e-01, -6.74489750e-01],
       [-6.74489750e-01,  6.74489750e-01,  6.74489750e-01, ...,
        -6.74489750e-01,  6.74489750e-01,  6.74489750e-01],
       ...,
       [ 6.74491251e-01,  7.76449228e-01, -1.00333038e+00, ...,
        -7.73347503e-01, -2.40735689e+00, -5.17291320e-01],
       [ 1.19525350e-06, -1.86261670e+00,  1.32763561e+00, ...,
        -7.61286466e-02,  6.99993215e-01,  1.31920482e-01],
       [-4.90096421e+00,  7.84327946e-02, -2.33077167e-01, ...,
         1.87579634e+00, -6.49417650e-01, -1.62130235e+00]])

In [59]:
# compute correlated samples
portfolio_simulated_risk_changes = (portfolio_cholesky @ portfolio_simulated_risk_changes.T).T
portfolio_simulated_risk_changes

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.74489750e-01,  4.07605678e-01,  1.77579205e-01, ...,
         1.02914060e+00, -8.01973046e-02, -1.40447852e-01],
       [-6.74489750e-01, -4.07605678e-01, -1.77579205e-01, ...,
        -1.02914060e+00,  8.01973046e-02,  1.40447852e-01],
       ...,
       [ 6.74491251e-01,  8.96773924e-01,  8.56986920e-01, ...,
        -1.54329328e+00, -2.89625803e+00, -1.35058354e+00],
       [ 1.19525350e-06, -6.27957961e-01, -7.74720990e-01, ...,
         2.79883719e-02,  7.85558341e-01,  3.84396329e-01],
       [-4.90096421e+00, -4.58759555e+00, -4.21867046e+00, ...,
         2.57590585e+00, -5.18650004e-01, -2.80038253e+00]])

In [60]:
# unstandardize correlated samples to distribution of portfolio
portfolio_simulated_risk_changes = portfolio_simulated_risk_changes * portfolio_risk_changes_std + portfolio_risk_changes_mean
portfolio_simulated_risk_changes

array([[1.834251998399881e-05, 6.5764790088367006e-06,
        6.707498381593196e-06, ..., 0.0016057696059283524,
        -0.0007383992089266441, -0.0011807395333843654],
       [0.0005599471625746949, 0.00037924984527873374,
        0.00016320382088627703, ..., 0.020053699065785743,
        -0.002549015310022004, -0.003486976891583095],
       [-0.0005232621226066972, -0.0003660968872610603,
        -0.00014978882412309066, ..., -0.016842159853929037,
        0.001072216892168716, 0.001125497824814364],
       ...,
       [0.0005599483674870516, 0.000826495765959575,
        0.0007619496550986875, ..., -0.02605863860640676,
        -0.06612727283881473, -0.023358125335563893],
       [1.834347975373021e-05, -0.00056756470573608,
        -0.0006760356537432704, ..., 0.0021074770603110877,
        0.016997166687302973, 0.005131277067774469],
       [-0.003917054522642727, -0.004187856441347997,
        -0.003711106275714794, ..., 0.04778034423706542,
        -0.012447970410623106, -0.04

In [61]:
# Stock simulated returns
# Stocks are the last 4 columns
stock_simulated_returns = portfolio_simulated_risk_changes[:, 10:]
stock_simulated_returns

array([[0.00046080236288395884, 0.0016057696059283524,
        -0.0007383992089266441, -0.0011807395333843654],
       [0.013616041530125479, 0.020053699065785743,
        -0.002549015310022004, -0.003486976891583095],
       [-0.012694436804357561, -0.016842159853929037,
        0.001072216892168716, 0.001125497824814364],
       ...,
       [-0.020983157534746215, -0.02605863860640676,
        -0.06612727283881473, -0.023358125335563893],
       [0.0007129158790264845, 0.0021074770603110877,
        0.016997166687302973, 0.005131277067774469],
       [0.018543454459645253, 0.04778034423706542, -0.012447970410623106,
        -0.047164688070459565]], dtype=object)

In [62]:
# SOFR simulated rate changes
# SOFR are the first 10 columns
sofr_simulated = portfolio_simulated_risk_changes[:, :10]
sofr_simulated

array([[1.834251998399881e-05, 6.5764790088367006e-06,
        6.707498381593196e-06, ..., 2.134761056332942e-05,
        2.2617222309097604e-05, 2.3682717698658473e-05],
       [0.0005599471625746949, 0.00037924984527873374,
        0.00016320382088627703, ..., -0.00010257063593001542,
        -0.00010872580319574528, -0.00010674974894454053],
       [-0.0005232621226066972, -0.0003660968872610603,
        -0.00014978882412309066, ..., 0.00014526585705667427,
        0.00015396024781394048, 0.00015411518434185748],
       ...,
       [0.0005599483674870516, 0.000826495765959575,
        0.0007619496550986875, ..., 0.0007802346220090097,
        0.0007950646901278664, 0.0008080574668498592],
       [1.834347975373021e-05, -0.00056756470573608,
        -0.0006760356537432704, ..., -0.0003826232820118393,
        -0.00035569236981244137, -0.00032751596808656123],
       [-0.003917054522642727, -0.004187856441347997,
        -0.003711106275714794, ..., -0.002479137032943146,
        -0.00

### <a id = "p5a">5a.</a>  <font color = "green"> Monte Carlo Full Revaluation </font>  [back to table of contents](#top)

In [63]:
# today's sofr
sofr_today = SOFR.iloc[-1, ].to_numpy()

# apply changes in risk factors to today's sofr
new_DF = (sofr_today + sofr_simulated).T

# compute discount factor for each time period
for i in range(len(new_DF)):
    new_DF[i] = np.exp(- new_DF[i].astype(float) * (i + 1))

n = len(sofr_simulated)
MC_full_reval_payer = np.zeros(n)

# compute payer swap value for each risk factor
for i in range(n):
    MC_full_reval_payer[i] = PV_payer_swap(new_DF.T[i])

# calculate change in swap value
MC_full_reval_payer = MC_full_reval_payer - payer_swap_today

MC_full_reval_payer_VaR = abs(np.percentile(MC_full_reval_payer, VaR_percentile))

print(f'Monte Carlo Full Revaluation Payer SOFR Swap 95% VaR: ${round(MC_full_reval_payer_VaR, 2):,}')

Monte Carlo Full Revaluation Payer SOFR Swap 95% VaR: $942,248.4


In [64]:
stock_simulated_returns

array([[0.00046080236288395884, 0.0016057696059283524,
        -0.0007383992089266441, -0.0011807395333843654],
       [0.013616041530125479, 0.020053699065785743,
        -0.002549015310022004, -0.003486976891583095],
       [-0.012694436804357561, -0.016842159853929037,
        0.001072216892168716, 0.001125497824814364],
       ...,
       [-0.020983157534746215, -0.02605863860640676,
        -0.06612727283881473, -0.023358125335563893],
       [0.0007129158790264845, 0.0021074770603110877,
        0.016997166687302973, 0.005131277067774469],
       [0.018543454459645253, 0.04778034423706542, -0.012447970410623106,
        -0.047164688070459565]], dtype=object)

In [65]:
# apply risk factor to today's prices to get new stock prices
MC_full_reval_stock_port = stocks_holdings_value * (1 + stock_simulated_returns)
MC_full_reval_stock_port

array([[1000460.802362884, 1001605.7696059284, 999261.6007910734,
        998819.2604666157],
       [1013616.0415301254, 1020053.6990657859, 997450.984689978,
        996513.0231084169],
       [987305.5631956424, 983157.840146071, 1001072.2168921686,
        1001125.4978248143],
       ...,
       [979016.8424652538, 973941.3613935933, 933872.7271611854,
        976641.8746644361],
       [1000712.9158790264, 1002107.4770603111, 1016997.166687303,
        1005131.2770677744],
       [1018543.4544596452, 1047780.3442370654, 987552.0295893769,
        952835.3119295404]], dtype=object)

In [66]:
# # multiply number of shares with stock price to get new stock portfolio value
# MC_full_reval_stock_port = stocks_holdings_value * MC_full_reval_prices
# MC_full_reval_stock_port

In [67]:
# subtract new stock portfolio value from current value 
MC_full_reval_stock_port_change = (MC_full_reval_stock_port - stocks_holdings_value).sum(axis = 1)
MC_full_reval_stock_port_change

array([147.43322650156915, 27633.748394306167, -27338.88194130361, ...,
       -136527.19431553152, 24948.83669441496, 6711.140215627849],
      dtype=object)

In [68]:
# 5% VaR for stock
MC_full_reval_VaR = abs(np.percentile(MC_full_reval_stock_port_change, VaR_percentile))
MC_full_reval_VaR

88129.57274606329

In [69]:
print(f'Monte Carlo Full Revaluation Stock 95% VaR: ${round(MC_full_reval_VaR, 2):,}')

Monte Carlo Full Revaluation Stock 95% VaR: $88,129.57


In [70]:
# sum portfolio changes due to swap and stock
MC_full_reval_portfolio_change = MC_full_reval_payer + MC_full_reval_stock_port_change
MC_full_reval_portfolio_change

array([18492.57540017739, -46358.28803407692, 83215.15861925855, ...,
       511308.36204700987, -263195.90326854936, -1907015.3111536242],
      dtype=object)

In [71]:
# 5th percentile for VaR
MC_full_reval_portfolio_VaR = abs(np.percentile(MC_full_reval_portfolio_change, VaR_percentile))
MC_full_reval_portfolio_VaR 

942527.0317546958

In [72]:
print(f'Monte Carlo Full Revaluation Portfolio 95% VaR: ${round(MC_full_reval_portfolio_VaR, 2):,}')

#plot(MC_full_reval_portfolio_change, MC_full_reval_portfolio_VaR, 'Monte Carlo Full Revaluation Portfolio Delta')

Monte Carlo Full Revaluation Portfolio 95% VaR: $942,527.03


### <a id = "p5b">5b.</a>  <font color = "green"> Monte Carlo Sensitivity Based Revaluation </font>  [back to table of contents](#top)

In [73]:
# compute payer swap value for each risk factor using PV01
MC_risk_based_payer = (pv01 * sofr_simulated).sum(axis = 1)

# 5th percentile for VaR
MC_risk_based_payer_VaR = abs(np.percentile(MC_risk_based_payer, VaR_percentile))

print(f'Monte Carlo Risk-Based Payer SOFR Swap 95% VaR: ${round(MC_risk_based_payer_VaR, 2):,}')

Monte Carlo Risk-Based Payer SOFR Swap 95% VaR: $936,811.52


In [74]:
# calculate change as a result of risk factor to the portfolio value
MC_risk_based_stock_port_change = stocks_holdings_value * stock_simulated_returns 
MC_risk_based_stock_port_change

array([[460.80236288395884, 1605.7696059283523, -738.3992089266442,
        -1180.7395333843654],
       [13616.041530125478, 20053.699065785742, -2549.015310022004,
        -3486.976891583095],
       [-12694.436804357561, -16842.15985392904, 1072.216892168716,
        1125.497824814364],
       ...,
       [-20983.157534746213, -26058.63860640676, -66127.27283881472,
        -23358.125335563895],
       [712.9158790264845, 2107.477060311088, 16997.16668730297,
        5131.277067774469],
       [18543.454459645254, 47780.344237065416, -12447.970410623106,
        -47164.68807045957]], dtype=object)

In [75]:
MC_risk_based_stock_port_change = MC_risk_based_stock_port_change.sum(axis = 1)
MC_risk_based_stock_port_change

array([147.43322650130176, 27633.748394306123, -27338.881941303523, ...,
       -136527.19431553158, 24948.836694415015, 6711.140215627995],
      dtype=object)

In [76]:
# 5% VaR for stock
MC_risk_based_VaR = abs(np.percentile(MC_risk_based_stock_port_change, VaR_percentile))
MC_risk_based_VaR

88129.5727460634

In [77]:
print(f'Monte Carlo Risk-Based Stock 95% VaR: ${round(MC_risk_based_VaR, 2):,}')

#plot(MC_risk_based_stock_port_change, MC_risk_based_VaR, 'Monte Carlo Risk-Based Stock Delta')

Monte Carlo Risk-Based Stock 95% VaR: $88,129.57


In [78]:
# sum portfolio changes due to swap and stock
MC_risk_based_portfolio_change = MC_risk_based_payer + MC_risk_based_stock_port_change
MC_risk_based_portfolio_change

array([18485.851627435295, -46275.82637288536, 83247.52962775592, ...,
       513442.701400603, -262612.62699387636, -1886458.6274244252],
      dtype=object)

In [79]:
# 5th percentile for VaR
MC_risk_based_portfolio_VaR = abs(np.percentile(MC_risk_based_portfolio_change, VaR_percentile))
MC_risk_based_portfolio_VaR

937077.3727554615

In [80]:
print(f'Monte Carlo Risk-Based Portfolio 95% VaR: ${round(MC_risk_based_portfolio_VaR, 2):,}')
#plot(MC_risk_based_portfolio_change, MC_risk_based_portfolio_VaR, 'Monte Carlo Risk-Based Portfolio Delta')

Monte Carlo Risk-Based Portfolio 95% VaR: $937,077.37


# <a id = "p6">6.</a>  <font color = "green"> Historical VaR Model </font>  [back to table of contents](#top)

### <a id = "p6a">6a.</a>  <font color = "green"> Historical Full Revaluation </font>  [back to table of contents](#top)

In [81]:
# sofr rate today
sofr_today = SOFR.iloc[-1, ].to_numpy()
sofr_today

array([0.0522446062245015, 0.0479042005661318, 0.0454293376025913,
       0.0443447502563426, 0.0439279746825301, 0.043794001970382,
       0.0437793521534213, 0.0438283144768502, 0.0439145618623502,
       0.044023149444895], dtype=object)

In [82]:
# apply historical rate changes to today's rates
new_DF = pd.DataFrame(sofr_today + sofr_delta.to_numpy())
new_DF

,0,1,2,3,4,5,6,7,8,9
0,0.052766,0.048343,0.045771,0.044641,0.044153,0.043918,0.043808,0.043785,0.043817,0.043879
1,0.052478,0.048378,0.045834,0.044599,0.044081,0.043912,0.043888,0.043927,0.044005,0.044115
2,0.052935,0.049002,0.046514,0.045302,0.04476,0.044555,0.044483,0.044452,0.044459,0.044518
3,0.051851,0.047408,0.044981,0.044022,0.043724,0.043678,0.043741,0.043874,0.04404,0.044215
4,0.052674,0.04844,0.045928,0.044758,0.04426,0.044107,0.044108,0.04417,0.04426,0.044362
...,...,...,...,...,...,...,...,...,...,...
245,0.052359,0.048276,0.045846,0.044687,0.044185,0.043959,0.043863,0.04386,0.043915,0.043996
246,0.052395,0.048297,0.046025,0.045088,0.044822,0.044775,0.044801,0.04488,0.044992,0.045123
247,0.051835,0.047157,0.04448,0.04329,0.042814,0.04267,0.042677,0.042756,0.042868,0.042995
248,0.052116,0.047618,0.045069,0.044006,0.043604,0.043537,0.043614,0.043736,0.043878,0.044034


In [83]:
# compute new discount factors for each period
for i in range(len(new_DF.columns)):
    new_DF[new_DF.columns[i]] = np.exp(- new_DF[new_DF.columns[i]].astype(float) * (i + 1))

new_DF = new_DF.to_numpy()
new_DF

array([[0.94860239, 0.90784075, 0.87169674, ..., 0.70448882, 0.67411413,
        0.6448137 ],
       [0.94887485, 0.90777713, 0.87153304, ..., 0.70369105, 0.67297937,
        0.6432989 ],
       [0.94844176, 0.90664502, 0.86975653, ..., 0.70074026, 0.6702309 ,
        0.64070583],
       ...,
       [0.94948592, 0.90999713, 0.87508108, ..., 0.71031596, 0.67989631,
        0.65054002],
       [0.94921895, 0.90915866, 0.87353605, ..., 0.70476877, 0.67374677,
        0.64381972],
       [0.94897333, 0.90837159, 0.87221145, ..., 0.70353616, 0.67278546,
        0.64314929]])

In [84]:
n = len(sofr_delta)
HS_full_reval_payer = np.zeros(n)
HS_full_reval_payer

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [85]:
# compute payer swap value for each risk factor
for i in range(n):
    HS_full_reval_payer[i] = PV_payer_swap(new_DF[i])

In [86]:
# calculate change in value of swap
HS_full_reval_payer = HS_full_reval_payer - payer_swap_today
HS_full_reval_payer

array([  -79854.56466062,    85728.00273665,   440438.32198618,
         111739.07747985,   279960.10813918,  -711893.65801421,
          47993.53171309, -2459267.73261779,   182761.21583111,
        -619010.52972043,  -951827.61645795,   668630.45827587,
         435154.73433946,   117612.72064245,  -751451.81044592,
        -364115.61116172,   -49693.23961608,   -46994.05137258,
         294766.92075293,  -426548.51402668, -1339805.24624082,
        -167024.61604358,   815479.61077196,  -852606.32560823,
        -699411.09477102,   783371.92852072,   634360.21180057,
         309218.19074896,  -925792.55580676,    32481.39165584,
        -305495.06718171,   226683.31568826,   720249.28581961,
         599348.93142374,    89424.99841706,   -22556.68604662,
         590202.54836401,   744154.64061143,   291904.74960852,
        -341393.22717476,  -193858.88575992,  -384772.55491956,
        -586933.7629523 ,   121292.75653612, -1209550.82567886,
        -443400.42447746,   770880.73529

In [87]:
HS_full_reval_payer_VaR = abs(np.percentile(HS_full_reval_payer, VaR_percentile))
HS_full_reval_payer_VaR

962541.6780712918

In [88]:
print(f'Historical Full Revaluation Payer SOFR Swap 95% VaR: ${round(HS_full_reval_payer_VaR, 2):,}')
#plot(HS_full_reval_payer, HS_full_reval_payer_VaR, 'Historical Full Revaluation Payer SOFR Swap Delta')

Historical Full Revaluation Payer SOFR Swap 95% VaR: $962,541.68


In [89]:
# apply risk factor to today's prices to get new stock prices
HS_full_reval_stock_port  = stocks_holdings_value * (1 + Stock_returns)
HS_full_reval_stock_port 

,AAPL,MSFT,FORD,BAC
Date,,,,
2022-11-01,9.824573e+05,9.829406e+05,1.002244e+06,1.004439e+06
2022-11-02,9.626949e+05,9.646316e+05,9.746270e+05,9.969612e+05
2022-11-03,9.575950e+05,9.734212e+05,1.015314e+06,9.944584e+05
2022-11-04,9.980526e+05,1.033326e+06,1.018854e+06,1.025077e+06
2022-11-07,1.003902e+06,1.029270e+06,1.014064e+06,1.005980e+06
...,...,...,...,...
2023-10-24,1.002543e+06,1.003674e+06,9.921604e+05,9.960891e+05
2023-10-25,9.865084e+05,1.030678e+06,1.013169e+06,1.003141e+06
2023-10-26,9.753945e+05,9.624857e+05,9.835356e+05,1.022309e+06


In [90]:
# # multiply shares with stock price to get new stock portfolio value
# HS_full_reval_stock_port = share_holdings * HS_full_reval_prices
# HS_full_reval_stock_port 

In [91]:
# subtract new stock portfolio value from current value 
HS_full_reval_stock_port_change = (HS_full_reval_stock_port - stocks_holdings_value).sum(axis = 1) 
HS_full_reval_stock_port_change

Date
2022-11-01    -27918.802772
2022-11-02   -101085.287174
2022-11-03    -59211.453965
2022-11-04     75308.487314
2022-11-07     53215.227421
                  ...      
2023-10-24     -5532.904106
2023-10-25     33496.788055
2023-10-26    -56275.090663
2023-10-27   -145012.650226
2023-10-30     36628.754030
Length: 250, dtype: float64

In [92]:
HS_full_reval_VaR = abs(np.percentile(HS_full_reval_stock_port_change, VaR_percentile))
HS_full_reval_VaR

82934.70634278827

In [93]:
print(f'Historical Full Revaluation Stock 95% VaR: ${round(HS_full_reval_VaR, 2):,}')
#plot(HS_full_reval_stock_port_change, HS_full_reval_VaR, 'Historical Full Revaluation Stock Delta')

Historical Full Revaluation Stock 95% VaR: $82,934.71


In [94]:
# sum portfolio changes due to swap and stock
HS_full_reval_portfolio_change = HS_full_reval_payer + HS_full_reval_stock_port_change
HS_full_reval_portfolio_change

Date
2022-11-01   -107773.367433
2022-11-02    -15357.284437
2022-11-03    381226.868022
2022-11-04    187047.564794
2022-11-07    333175.335560
                  ...      
2023-10-24     -1018.381280
2023-10-25    896247.281706
2023-10-26   -899044.674491
2023-10-27   -166565.411089
2023-10-30    133719.175468
Length: 250, dtype: float64

In [95]:
# 5th percentile for VaR
HS_full_reval_portfolio_VaR = abs(np.percentile(HS_full_reval_portfolio_change, VaR_percentile))
HS_full_reval_portfolio_VaR

966173.2175561615

In [96]:
print(f'Historical Full Revaluation Portfolio 95% VaR: ${round(HS_full_reval_portfolio_VaR, 2):,}')
#plot(HS_full_reval_portfolio_change, HS_full_reval_portfolio_VaR, 'Historical Full Revaluation Portfolio Delta')

Historical Full Revaluation Portfolio 95% VaR: $966,173.22


### <a id = "p6b">6b.</a>  <font color = "green"> Historical Sensitivity Based Revaluation </font>  [back to table of contents](#top)

In [97]:
# compute payer swap value for each risk factor using PV01
HS_risk_based_payer = (pv01 * sofr_delta.to_numpy()).sum(axis = 1).astype(float)
HS_risk_based_payer

array([  -79728.425997  ,    85730.34607089,   441289.24901396,
         111813.54484793,   280271.39651024,  -708618.01485979,
          48028.65273853, -2424086.78861718,   182861.36228079,
        -616547.13107815,  -945796.91878382,   670874.22905657,
         436005.39645682,   117635.70075827,  -747725.23369791,
        -363165.48706719,   -49654.27125848,   -46959.365003  ,
         295189.77443782,  -425381.01507957, -1328695.35752807,
        -166775.90877183,   818872.3400575 ,  -847749.9484202 ,
        -696329.29792891,   786623.18526317,   636514.18162344,
         309613.61418248,  -920619.26147415,    32472.89361275,
        -304755.32998627,   226957.4953531 ,   722968.45595813,
         601211.80712389,    89438.28709794,   -22523.42042128,
         591979.02921621,   747047.19385619,   292298.85171193,
        -340486.74041809,  -193521.71543039,  -383702.86820038,
        -584609.88065461,   121321.80266487, -1200907.51262084,
        -442067.16589195,   774096.77503

In [98]:
# 5th percentile for VaR
HS_risk_based_payer_VaR = abs(np.percentile(HS_risk_based_payer, VaR_percentile))
HS_risk_based_payer_VaR

956909.6672652599

In [99]:
print(f'Historical Risk-Based Payer SOFR Swap 95% VaR: ${round(HS_risk_based_payer_VaR, 2):,}')
#plot(HS_risk_based_payer, HS_risk_based_payer_VaR, 'Historical Risk-Based Payer SOFR Swap Delta')

Historical Risk-Based Payer SOFR Swap 95% VaR: $956,909.67


In [100]:
# immediately multiply change in risk to holdings
HS_risk_based_stock_port_change = stocks_holdings_value * Stock_returns
HS_risk_based_stock_port_change

,AAPL,MSFT,FORD,BAC
Date,,,,
2022-11-01,-17542.655335,-17059.428567,2243.790493,4439.490637
2022-11-02,-37305.060428,-35368.363675,-25373.027872,-3038.835199
2022-11-03,-42404.984693,-26578.818853,15313.925077,-5541.575496
2022-11-04,-1947.356312,33325.554932,18853.703516,25076.585179
2022-11-07,3902.093532,29269.692159,14063.576596,5979.865134
...,...,...,...,...
2023-10-24,2543.334713,3674.240518,-7839.614216,-3910.865120
2023-10-25,-13491.581744,30678.046863,13169.354821,3140.968115
2023-10-26,-24605.546085,-37514.323638,-16464.393007,22309.172066


In [101]:
# sum change
HS_risk_based_stock_port_change = HS_risk_based_stock_port_change.sum(axis = 1)
HS_risk_based_stock_port_change

Date
2022-11-01    -27918.802772
2022-11-02   -101085.287174
2022-11-03    -59211.453965
2022-11-04     75308.487314
2022-11-07     53215.227421
                  ...      
2023-10-24     -5532.904106
2023-10-25     33496.788055
2023-10-26    -56275.090663
2023-10-27   -145012.650226
2023-10-30     36628.754030
Length: 250, dtype: float64

In [102]:
# 5th percentile for VaR
HS_risk_based_VaR = abs(np.percentile(HS_risk_based_stock_port_change, VaR_percentile))
HS_risk_based_VaR

82934.70634278834

In [103]:
print(f'Historical Risk-Based Stock 95% VaR: ${round(HS_full_reval_VaR, 2):,}')
#plot(HS_risk_based_stock_port_change, HS_risk_based_VaR, 'Historical Risk-Based Stock Delta')

Historical Risk-Based Stock 95% VaR: $82,934.71


In [104]:
# sum portfolio changes due to swap and stock
HS_risk_based_portfolio_change = HS_risk_based_payer + HS_risk_based_stock_port_change
HS_risk_based_portfolio_change

Date
2022-11-01   -107647.228769
2022-11-02    -15354.941103
2022-11-03    382077.795049
2022-11-04    187122.032162
2022-11-07    333486.623931
                  ...      
2023-10-24      -999.474390
2023-10-25    900308.581935
2023-10-26   -894592.280498
2023-10-27   -166544.013928
2023-10-30    133727.175245
Length: 250, dtype: float64

In [105]:
# 5th percentile for VaR
HS_risk_based_portfolio_VaR = abs(np.percentile(HS_risk_based_portfolio_change, VaR_percentile))
HS_risk_based_portfolio_VaR 

960687.7194580392

In [106]:
print(f'Historical Risk-Based Portfolio 95% VaR: ${round(HS_risk_based_portfolio_VaR, 2):,}')
#plot(HS_risk_based_portfolio_change, HS_risk_based_portfolio_VaR, 'Historical Risk-Based Portfolio Delta')

Historical Risk-Based Portfolio 95% VaR: $960,687.72


In [107]:
print(f'Parametric VaR${round(Para_port_VaR, 2):,}')
print(f'Monte Carlo Full Revaluation ${round(MC_full_reval_portfolio_VaR, 2):,}')
print(f'Monte Carlo Sensitivity-based ${round(MC_risk_based_portfolio_VaR, 2):,}')
print(f'Historical Full Revaluation ${round(HS_full_reval_portfolio_VaR, 2):,}')
print(f'Historical Sensitivity-based ${round(HS_risk_based_portfolio_VaR, 2):,}')

Parametric VaR$936,889.26
Monte Carlo Full Revaluation $942,527.03
Monte Carlo Sensitivity-based $937,077.37
Historical Full Revaluation $966,173.22
Historical Sensitivity-based $960,687.72
